<a href="https://colab.research.google.com/github/georgeaidinis/NTUA/blob/master/NN/Lab%201/Copy_of_Lab_1_Small_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Αϊδίνης* Γιώργος 03116031

Κολιός Παναγιώτης 03116100

---

Ομάδα M.B.8

---


#Lab 1: Επιβλεπόμενη Μάθηση - Ταξινόμηση - Μικρό Dataset (S11 - Quality Assessment of Digital Colposcopies)

Αρχικά ενημερώνουμε τις βιβλιοθήκες που θα χρησιμοποιήσουμε.

In [1]:
!pip install --upgrade pip #upgrade pip package installer
!pip install scikit-learn --upgrade #upgrade scikit-learn package
!pip install numpy --upgrade #upgrade numpy package
!pip install pandas --upgrade #--upgrade #upgrade pandas package

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 7.0MB 9.1MB/s 
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3
Requirement already up-to-date: numpy in /usr/local/lib/python3.6/dist-packages (1.17.4)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (0.25.3)


1. Το dataset περιγράφει μετρήσεις κολποσκοπήσεων και την κατάσταση (ετικέτα) των αντίστοιχων κόλπων, όπως προκύπτει από τις εκτιμήσεις καθενός από 6 ειδικούς και την εκτίμηση της πλειοψηφίας. Οι μετρήσεις και οι εκτιμήσεις είναι διαφορετικές για τις διαφορετικές μεθόδους με τις οποίες πραγματοποιούνται οι κολποσκοπήσεις: Hinselmann, Green, Schiller. 

2. Έχουμε συνολικά 287 δείγματα, 92 από τη μέθοδο Schiller, 98 από τη μέθοδο Green και 97 από τη μέθοδο Hinselmann. <br>
Σε κάθε δείγμα περιέχονται μετρήσεις για 62 χαρακτηριστικά-παρατηρήσεις από τα οποία προέκυψαν οι εκτιμήσεις. Όλα τα χαρακτηριστικά είναι διατεταγμένα.<br>
Οι μετρήσεις είναι αριθμητικά δεδομένα και αφορούν τιμές όπως τις επιφάνειες περιοχών του κόπλου. 

3. Υπάρχουν επικεφαλίδες στην πρώτη γραμμή πάνω από τα χαρακτηριστικά και τις ετικέτες, οι οποίες θα πρέπει να αφαιρεθούν. Δεν υπάρχει στήλη για την αρίθμηση των γραμμών.<br>

4. Η τιμή της κατάστασης μπορεί να πάρει δύο τιμές, 0 για κακή και 1 για καλή. Όπως υποδεικνύεται από τις FAQ το πρόβλημα θα αναλυθεί ως binary classification λαμβάνοντας υπόψην μόνο τις εκτιμήσεις της πλειοψηφίας. Έτσι οι στήλες των ετικετών που αφορούν μεμονωμένα τον κάθε ειδικό αφαιρούνται και μένει μόνο η τελευταία στήλη με τις εκτιμήσεις της πλειοψηφίας. Η στήλη αυτή είναι η τελευταία (θέση 69 στον αρχικό πίνακα). Παρατηρούμε οτι όλες οι ισοψηφίες (3-3) επιλύονται θεωρώντας την κατάσταση ως καλή, γεγονός που ίσως μας αναγκάσει να αυξήσουμε αργότερα το πλήθος των δειγμάτων με ετικέτα 0 ή να διαγράψουμε δείγματα με ετικέτα 1.

5. Συνενώνουμε τα αρχεία που αφορούν τις 3 διαφορετικές μεθόδους. Μετά τη συνένωση οι επικεφαλίδες προστέθηκαν ως γραμμές, με αποτέλεσμα να έχουμε 3 φορές την ίδια γραμμή με τις επικεφαλίδες, τις οποίες και αφαιρούμε.

6. Δεν υπάρχουν απουσιάζουσες τιμές.

7. 

In [0]:
from urllib.request import urlretrieve
import os
from zipfile import ZipFile

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")


In [14]:
download("http://archive.ics.uci.edu/ml/machine-learning-databases/00384/Quality%20Assessment%20-%20Digital%20Colposcopy.zip","QADC.zip")
print("All the files are downloaded")

All the files are downloaded


In [15]:
!ls

 all.csv    Quality_Assessment_Digital_Colonoscopy.zip	 sample_data
 QADC.zip  'Quality Assessment - Digital Colposcopy'


In [16]:
!unzip -q "QADC.zip"

replace Quality Assessment - Digital Colposcopy/schiller.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Quality Assessment - Digital Colposcopy/hinselmann.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Quality Assessment - Digital Colposcopy/green.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Quality Assessment - Digital Colposcopy/images/vaginal walls/schiller/76.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Quality Assessment - Digital Colposcopy/images/vaginal walls/schiller/77.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Quality Assessment - Digital Colposcopy/images/vaginal walls/schiller/78.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Quality Assessment - Digital Colposcopy/images/vaginal walls/schiller/79.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Quality Assessment - Digital Colposcopy/images/vaginal walls/schiller/80.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Quality Assessment - Digital Colposcopy/images/vaginal wa

In [0]:
!cat Quality\ Assessment\ -\ Digital\ Colposcopy/green.csv Quality\ Assessment\ -\ Digital\ Colposcopy/hinselmann.csv Quality\ Assessment\ -\ Digital\ Colposcopy/schiller.csv > all.csv